In [1]:
import random

import os
import shutil
import numba

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from scipy import linalg
from scipy import fftpack
from scipy import stats
from sklearn.preprocessing import normalize, MinMaxScaler
import scipy.signal as sgn
import scipy.misc

import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.vectors import IntVector, FloatVector
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
# Define functions
# import transfer entropy function from R:
TXE = rpackages.importr("RTransferEntropy")
def TE_matrix(matrix, memory=20):
    n = np.shape(matrix)[1]
    TE = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i != j:
                TE[i,j] = TXE.transfer_entropy(IntVector(matrix[:,i]), IntVector(matrix[:,j]), 
                                               lx = memory, ly = memory,  
                                               entropy = "Shannon", 
                                               q = 1,
                                              quiet=True)[2][0]
    return(TE)


# calculate distance matrix from position matrix
def distanceMatrix(plist):
    n = np.shape(plist)[0]
    dmat = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            dmat[i,j] = np.linalg.norm(POSITIONS[i]-POSITIONS[j])
    return(dmat)

# calculate undirected adjacency matrix
def connectionMatrix(fmat, nlist):
    n = np.shape(fmat)[1]
    cmat = np.zeros((n,n))
    for connection in nlist:
        nfrom = connection[0] - 1
        nto = connection[1] - 1
        if connection[2] != 0:
            cmat[nfrom, nto] = 1
            cmat[nto, nfrom] = 1
    return(cmat)

# calculate directed adjacency matrix
def projectionMatrix(fmat, nlist):
    n = np.shape(fmat)[1]
    pmat = np.zeros((n,n))
    for connection in nlist:
        nfrom = connection[0] - 1
        nto = connection[1] - 1
        ctype = connection[2]
        cmat[nfrom, nto] = ctype
    return(pmat)

def spike_detect(flr, h=0.125, w=10):
    s = np.zeros(np.shape(flr))
    for j in range(np.shape(flr)[1]):
        t = sgn.find_peaks(flr[:,j],
                           h,
                           width=w) 
        for i in t[0]:
            s[i,j] = 1
    return(s)

def hp_1(track):
    
    # attempt hp on 2d matrix
    try:
        new_track = np.zeros((np.shape(track)[0], np.shape(track)[1]))
        for i in range(np.shape(track)[0]):
            for j in range(np.shape(track)[1]):
                try:
                    new_track[i,j] = (track[i-1,j] + track[i,j] + track[i+1],j)     
                except:
                    new_track[i,j] = track[i,j]
    
    # if dimensional error, hp 1 dimensional
    except:       
        new_track = np.zeros(len(track))
        for i in range(len(track)):
            try:
                new_track[i] = (track[i-1] + track[i] + track[i+1])     
            except:
                new_track[i] = track[i]

    return(new_track)

def hp_2(track): 
    try:# attempt hp on 2d matrix
        new_track = np.zeros((np.shape(track)[0], np.shape(track)[1]))
        for i in range(np.shape(track)[0]):
            for j in range(np.shape(track)[1]):
                try:
                    new_track[i,j] = (0.4* track[i-3,j] + 0.6*track[i-2,j] + 0.8*track[i-1,j] + track[i,j])     
                except:
                    new_track[i,j] = track[i,j]       
    except:# if dimensional error, hp 1 dimensional
        new_track = np.zeros(len(track))
        for i in range(len(track)):
            try:
                new_track[i] = (0.4* track[i-3] + 0.6*track[i-2] + 0.8*track[i-1] + track[i])     
            except:
                new_track[i] = track[i]
    return(new_track)

def lp(track):
    try:# attempt lp on 2d matrix
        new_track = np.zeros((np.shape(track)[0], np.shape(track)[1]))
        for i in range(np.shape(track)[0]):
            for j in range(np.shape(track)[1]):
                try:
                    new_track[i,j] = track[i,j] - track[i-1,j]
                except:
                    new_track[i,j] = track[i,j]    
    except:
        new_track = np.zeros(len(track))
        for i in range(len(track)):
            try:
                new_track[i] = track[i] - track[i-1]
            except:
                new_track[i] = track[i]
    return(new_track)

def thresh(track, th=0.05):
    try:# attempt threshold on 2d matrix
        new_track = np.zeros((np.shape(track)[0], np.shape(track)[1]))
        for i in range(np.shape(track)[0]):
            for j in range(np.shape(track)[1]):
                if track[i,j] < th:
                    new_track[i,j] = 0
                else:
                    new_track[i,j] = track[i,j]
    except:# threshold 1d track
        new_track = np.zeros(len(track))
        for i in range(len(track)):
            if track[i] < th:
                new_track[i] = 0
            else:
                new_track[i] = track[i]
    return(new_track)

def all_filter(track):
    track = thresh(lp(hp_2(hp_1(track))))
    return(track)

def pc(mat):
    p = np.zeros((np.shape(mat)[1], np.shape(mat)[1]))
    prec = linalg.inv(np.cov(mat, rowvar = False))
    print(linalg.det(prec))
    for i in range((np.shape(p)[0])):
        for j in range((np.shape(p)[1])):
            p[i, j] = -1 * prec[i,j] / (np.sqrt(prec[i,i]*prec[j,j]))
    return(p)

def mutual_info(matrix, window=50):
    n = np.shape(matrix)[1]
    MImat = np.zeros((n,n))
    periods = []
    for i in range(0, int(np.floor(np.shape(matrix)[0] / window)-1)):
        periods.append([i*window, (i+1)*window])
            
    for i in range(0,n):
        for j in range(0,n):
            MI = []
            for p in periods:
                #print("i={}, j={},p={}".format(i,j,p))
                if i!=j:
                    info = mutual_info_classif(matrix[p[0]:p[1],i].reshape(-1,1), 
                                                  matrix[p[0]:p[1],j],
                                                  n_neighbors=5,
                                                  discrete_features=True)
                    MI.append(info)
                else:
                    MI.append(0)
                
                #print("{},{}: {}".format(i,j,MI))
            MImat[i,j] = (np.mean(MI))
    return(MImat)

def weight_filt(track):    
    new_track = np.zeros((np.shape(track)[0], np.shape(track)[1]))
    for i in range(np.shape(track)[0]):
        for j in range(np.shape(track)[1]):
            s = np.sum(track[i,:])
            if abs(s) < 0.01:
                new_track[i,j] = 1
            else:
                new_track[i,j] = (track[i,j] + 1) ** (1 + (1 / np.sum(track[i, :])))
    return(new_track)

def pc(mat):
    p = np.zeros((np.shape(mat)[1], np.shape(mat)[1]))
    prec = linalg.inv(np.cov(mat, rowvar = False))
    for i in range((np.shape(p)[0])):
        for j in range((np.shape(p)[1])):
            p[i, j] = -1 * prec[i,j] / (np.sqrt(prec[i,i]*prec[j,j]))
    return(p)

def mutual_info(matrix, window=50):
    n = np.shape(matrix)[1]
    MImat = np.zeros((n,n))
    periods = []
    for i in range(0, int(np.floor(np.shape(matrix)[0] / window)-1)):
        periods.append([i*window, (i+1)*window])
            
    for i in range(0,n):
        for j in range(0,n):
            MI = []
            for p in periods:
                #print("i={}, j={},p={}".format(i,j,p))
                if i!=j:
                    info = mutual_info_classif(matrix[p[0]:p[1],i].reshape(-1,1), 
                                                  matrix[p[0]:p[1],j],
                                                  n_neighbors=5, 
                                                  discrete_features=True)*1000
                    MI.append(info)
                else:
                    MI.append(0)
                
                #print("{},{}: {}".format(i,j,MI))
            MImat[i,j] = (np.quantile(MI,.75))
    return(MImat)

def grangerMatrix(matrix, mem = 5):
    # pass a spike train matrix
    n = np.shape(matrix)[1]
    gMat = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i!=j:
                f = []
                c = matrix[:,(i,j)]
                g = granger(c, maxlag = mem, verbose=False)
                for k in g.keys():
                    f.append(g[k][0]['ssr_ftest'][0])
                gMat[i,j] = max(f)
    return(gMat)



def upper_thresh(matrix, th = 15):
    for i in range(np.shape(matrix)[0]):
        for j in range(np.shape(matrix)[1]):
            if matrix[i,j]>th:
                matrix[i,j]=th
    return(matrix)

In [207]:
# Set up file paths
parent_path = "/Users/David/Documents/NYU/MACHINE_LEARNING/netconnect/"
source_path = "/Users/David/Documents/NYU/MACHINE_LEARNING/netconnect/src/"
data_path = "/Users/David/Documents/NYU/MACHINE_LEARNING/netconnect/data/"


ffile = "fluorescence_iNet1_Size100_CC06inh.txt"
nfile = "network_iNet1_Size100_CC06inh.txt"
pfile = 'networkPositions_iNet1_Size100_CC06inh.txt'

fpath = "{}{}".format(source_path, ffile)
npath = "{}{}".format(source_path, nfile)
ppath = "{}{}".format(source_path, pfile)


train_dir = "{}{}".format(data_path, "train/")
validation_dir = "{}{}".format(data_path, "validation/")
test_dir = "{}{}".format(data_path, "test/")

# Import data
FLR = np.genfromtxt(fpath,
                    delimiter=',')
NETWORK = np.genfromtxt(npath,
                        delimiter=',').astype(int)
POSITIONS = np.genfromtxt(ppath,
                          delimiter=',')

# high pass and low pass filters, scale to 0:1
fflr = lp(hp_2(hp_1(FLR)))
scaler = MinMaxScaler()
scale = scaler.fit(fflr)
sflr = scale.transform(fflr)
# weight by network activity
wflr = weight_filt(sflr)
# center weighted flr on mean
cflr = wflr - np.mean(wflr)
# threshold and spike detect
tflr = thresh(cflr, th=0.15)
spkflr = spike_detect(tflr, w = 0.1, h = 0.1)



tempout = "/Users/David/Desktop/output/"
# discrete and continuous partial correlation
discrete = pc(spkflr)
continuous = pc(cflr)

# save spikes
np.savetxt("{}SPIKES_06.csv".format(tempout), spkflr, delimiter = ",")
# save discrete pc
np.savetxt("{}PC_DISC_06.csv".format(tempout), discrete, delimiter = ",")
# save continuous pc
np.savetxt("{}PC_CONT_06.csv".format(tempout), continuous, delimiter = ",")

In [208]:
print("complete")

complete
